In [1]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text, desc, union, union_all, asc
from sqlalchemy.sql import func, exists, text, distinct
import pandas as pd
import os

In [6]:
engine = create_engine('sqlite+pysqlite:///Pilots_db.sqlite')
conn=engine.connect()

Задача 1. Напишите запрос, который выведет пилотов, которые в качестве второго пилота в
августе этого года трижды ездили в аэропорт Шереметьево. 

In [7]:
sql = '''
with subq AS(
    SELECT second_pilot_id 
    FROM Flight 
    WHERE destination ='Шереметьево' AND flight_dt like '%.08.2021' 
    GROUP BY second_pilot_id 
    HAVING COUNT(flight_id) = 3
)
SELECT *
FROM Pilot 
where pilot_id in subq 
'''

df = pd.read_sql(sql, conn)
print(df)

   pilot_id      name  age  rank  education_level
0        30  Аристарх   40     1               16


Задача 2. Выведите пилотов старше 45 лет, совершали полеты на самолетах с количеством
пассажиров больше 30.

In [8]:
sql = '''
WITH subq_civil_planes AS(
    SELECT plane_id 
    FROM Plane 
    WHERE cargo_flg=0
),
subq2 AS (
    SELECT first_pilot_id
    FROM Flight 
    WHERE plane_id IN subq_civil_planes
    AND quantity > 30
),
subq3 AS (
    SELECT second_pilot_id
    FROM Flight 
    WHERE plane_id IN subq_civil_planes
    AND quantity > 30
)

SELECT *
FROM Pilot
WHERE (pilot_id IN subq2 OR pilot_id IN subq3)
AND age > 45
'''
df = pd.read_sql(sql, conn)
print(df)

    pilot_id               name  age  rank  education_level
0          3               Аваз   52     2                3
1         11             Азарий   48     5                4
2         12               Аким   46     7                5
3         18             Амадей   53     3               15
4         20              Амаяк   51     5               16
5         26              Антон   48     3               17
6         45                Бег   50     1               15
7         54              Борис   48     4               19
8         27  Ануфрий (Онуфрий)   51     4                3
9         34               Арон   54     7               19
10        44                  Б   48     1               20
11        46      Бежен (Бажен)   46     4                8


Задача 3. Выведите ТОП 10 пилотов-капитанов (first_pilot_id), которые совершили наибольшее
число грузовых перелетов в этом году.

In [9]:
sql = '''
WITH subq_gruz_planes AS(
    SELECT plane_id 
    FROM Plane 
    WHERE cargo_flg=1
)

SELECT Pilot.*, count(flight_id) AS "Amount_of_flights"
FROM Pilot 
JOIN flight ON pilot.pilot_id = flight.first_pilot_id
WHERE plane_id in subq_gruz_planes
GROUP BY Pilot.pilot_id
ORDER BY Amount_of_flights DESC
LIMIT (10)
'''
df = pd.read_sql(sql, conn)
print(df)

   pilot_id      name  age  rank  education_level  Amount_of_flights
0        20     Амаяк   51     5               16                  3
1        59         В   35     6               14                  2
2        41  Афанасий   52     1                6                  2
3        19   Амадеус   37     5               12                  2
4        47  Бенедикт   36     2               15                  1
5        42     Ахмет   39     4               18                  1
6        37   Артемий   50     1               18                  1
7        34      Арон   54     7               19                  1
8        26     Антон   48     3               17                  1
9        24    Андрей   29     2               14                  1
